In [1]:
import pandas as pd
import numpy as np

In [2]:
numeric = "../input_orig/train_numeric.csv"
date = "../input_orig/train_date.csv"
categorical = "../input_orig/train_categorical.csv"

numeric_te = "../input_orig/test_numeric.csv"
date_te = "../input_orig/test_date.csv"
categorical_te = "../input_orig/test_categorical.csv"

In [3]:
#tr_line_station_minmax = pd.read_hdf('feats/tr_line_station_minmax8.hdf5','table')
#te_line_station_minmax = pd.read_hdf('feats/te_line_station_minmax8.hdf5','table')

In [4]:
def orgainize(features):
    line_features = {}
    station_features = {}
    lines = set([f.split('_')[0] for f in features])
    stations = set([f.split('_')[1] for f in features])
    
    for l in lines:
        line_features[l] = [f for f in features if l+'_' in f]
        
    for s in stations:
        station_features[s] = [f for f in features if s+'_' in f]
            
    return line_features, station_features

In [5]:
features_num = pd.read_csv(numeric, nrows=1).drop(['Response', 'Id'], axis=1).columns.values
line_features_num, station_features_num = orgainize(features_num)
print("Features in Station 32: {}".format( station_features_num['S32'] ))

Features in Station 32: ['L3_S32_F3850']


In [6]:
features_date = pd.read_csv(date, nrows=1).drop(['Id'], axis=1).columns.values
line_features_date, station_features_date = orgainize(features_date)
print("Features in Station 32: {}".format( station_features_date['S32'] ))

Features in Station 32: ['L3_S32_D3852']


In [7]:
features_cat = pd.read_csv(categorical, nrows=1).drop(['Id'], axis=1).columns.values
line_features_cat, station_features_cat = orgainize(features_cat)
print("Features in Station 32: {}".format( station_features_cat['S32'] ))

Features in Station 32: ['L3_S32_F3851', 'L3_S32_F3853', 'L3_S32_F3854']


In [8]:
CHUNKSIZE = 100000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

def make_features_date(df):
    new_features = pd.DataFrame({})
    for s in station_features_date.keys():
        station_data = df[station_features_date[s]]
        maxx = station_data.max(axis=1)
        minn = station_data.min(axis=1)
        col = s+'_diff_date'
        new_features[col] = maxx - minn
    return new_features

tr_station_diff_date = pd.DataFrame()
for tr in pd.read_csv(date, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(tr.columns, [ID_COLUMN])
    chunk_features = make_features_date(tr[feats])
    tr_station_diff_date = tr_station_diff_date.append(chunk_features)
    
    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break
        
te_station_diff_date = pd.DataFrame()
for te in pd.read_csv(date_te, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(te.columns, [ID_COLUMN])
    chunk_features = make_features_date(te[feats])
    te_station_diff_date = te_station_diff_date.append(chunk_features)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [9]:
tr_station_diff_date.head()

,S28_diff_date,S41_diff_date,S39_diff_date,S2_diff_date,S20_diff_date,S3_diff_date,S27_diff_date,S45_diff_date,S32_diff_date,S8_diff_date,...,S16_diff_date,S35_diff_date,S4_diff_date,S33_diff_date,S7_diff_date,S37_diff_date,S5_diff_date,S19_diff_date,S43_diff_date,S12_diff_date
0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN
3,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [10]:
te_station_diff_date.head()

,S28_diff_date,S41_diff_date,S39_diff_date,S2_diff_date,S20_diff_date,S3_diff_date,S27_diff_date,S45_diff_date,S32_diff_date,S8_diff_date,...,S16_diff_date,S35_diff_date,S4_diff_date,S33_diff_date,S7_diff_date,S37_diff_date,S5_diff_date,S19_diff_date,S43_diff_date,S12_diff_date
0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN


In [11]:
tr_station_diff_date.shape, te_station_diff_date.shape

((1183747, 52), (1183748, 52))

In [12]:
CHUNKSIZE = 100000
NROWS_TR = 1183747
nrows_tr = 0
NROWS_TE = 1183748
nrows_te = 0
ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
SEED = 0

def make_features_date(df):
    new_features = pd.DataFrame({})
    for s in line_features_date.keys():
        line_data = df[line_features_date[s]]
        maxx = line_data.max(axis=1)
        minn = line_data.min(axis=1)
        col = s+'_diff_date'
        new_features[col] = maxx - minn
    return new_features

tr_line_diff_date = pd.DataFrame()
for tr in pd.read_csv(date, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(tr.columns, [ID_COLUMN])
    chunk_features = make_features_date(tr[feats])
    tr_line_diff_date = tr_line_diff_date.append(chunk_features)
    
    nrows_tr += CHUNKSIZE
    if nrows_tr >= NROWS_TR:
        break
        
te_line_diff_date = pd.DataFrame()
for te in pd.read_csv(date_te, chunksize=CHUNKSIZE, dtype=np.float32):
    feats = np.setdiff1d(te.columns, [ID_COLUMN])
    chunk_features = make_features_date(te[feats])
    te_line_diff_date = te_line_diff_date.append(chunk_features)
    
    nrows_te += CHUNKSIZE
    if nrows_te >= NROWS_TE:
        break

In [13]:
te_line_diff_date.head()

,L1_diff_date,L0_diff_date,L3_diff_date,L2_diff_date
0,0.050049,NaN,0.049988,0.0
1,0.010010,NaN,0.029907,0.0
2,0.030029,NaN,0.140015,0.0
3,NaN,0.040009,0.029999,0.0
4,0.020020,NaN,0.010010,0.0


In [14]:
tr_line_diff_date.head()

,L1_diff_date,L0_diff_date,L3_diff_date,L2_diff_date
0,NaN,0.029999,0.029999,NaN
1,NaN,0.030029,0.020020,NaN
2,NaN,0.030029,0.020020,NaN
3,NaN,0.020020,0.040039,NaN
4,NaN,0.029968,0.020020,NaN


In [15]:
tr_line_diff_date.shape, te_line_diff_date.shape

((1183747, 4), (1183748, 4))

In [16]:
tr_line_station_diff_date = pd.concat([tr_station_diff_date,tr_line_diff_date],axis=1)
te_line_station_diff_date = pd.concat([te_station_diff_date,te_line_diff_date],axis=1)

In [17]:
tr_line_station_diff_date.to_hdf('feats/tr_line_station_diff_date10.hdf5','table')
te_line_station_diff_date.to_hdf('feats/te_line_station_diff_date10.hdf5','table')